## Recognizing Face and give Mail and Whatapp Alert ❗

### 🔻 Taking Samples of Face 1

In [1]:

# librarires
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user1/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\DILEEP~1\AppData\Local\Temp/ipykernel_12048/1137530677.py:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


### 🔻 Creating Model with the samples collected 

In [20]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user1/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

ganesh_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
ganesh_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

Model trained sucessefully


### 🔻 Creating Face Recognition Program

In [24]:
import cv2
import numpy as np
import os
import time
import pywhatkit as kit
import smtplib
from email.message import EmailMessage


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = ganesh_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            print(confidence)
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 84:
            cv2.putText(image, "Hey Ganesh", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
#             sending whatsapp message
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
            kit.sendwhatmsg_instantly(phone_no="+919110619797", message="Hey just now Ganesh tried to login to your system")
            print("\t\t\t Whatsapp message successfully sent")
            print("                                       >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
            print()
            print()
            
            
            # sending mail
            Sender_Email = "dileep14220021@gmail.com"
  
            Reciever_Email = "dileepkumar1422002@gmail.com"

            Password = "dileep@123"
    
            newMessage = EmailMessage()                         
            newMessage['Subject'] = "Alert Message." 
            newMessage['From'] = Sender_Email                   
            newMessage['To'] = Reciever_Email                   
            newMessage.set_content('🏮 Hey just now Ganesh tried to login to your system❗❗❗❗') 
        
        
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

                smtp.login(Sender_Email, Password)              
                smtp.send_message(newMessage) 

            print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>")
            print("\t\t\t Mail Successfully Sent")
            print("                                       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
            break
            
        elif confidence>60 & confidence<84 :
            cv2.putText(image, "Face Detected", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
#             sending whatsapp message
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
            kit.sendwhatmsg_instantly(phone_no="+919110619797", message="🏮 Hey some unauthorized user logins to the system ❗❗❗❗❗")
            print("\t\t\t Whatsapp message successfully sent")
            print("                                       >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
            print()
            print()
            
            
            # sending mail
            Sender_Email = "dileep14220021@gmail.com"
  
            Reciever_Email = "dileepkumar1422002@gmail.com"

            Password = "dileep@123"
    
            newMessage = EmailMessage()                         
            newMessage['Subject'] = "Alert Message." 
            newMessage['From'] = Sender_Email                   
            newMessage['To'] = Reciever_Email                   
            newMessage.set_content('🏮 Hey some unauthorized user tries to login to your system ❗❗❗❗❗') 
        
        
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

                smtp.login(Sender_Email, Password)              
                smtp.send_message(newMessage) 

            print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>")
            print("\t\t\t Mail Successfully Sent")
            print("                                       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
            break
            
         
        else:
            
            cv2.putText(image, "I dont know, who r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        

        
        
cap.release()
cv2.destroyAllWindows() 

<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\DILEEP~1\AppData\Local\Temp/ipykernel_6356/45258377.py:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
